In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict, Counter
import glob
import json
from random import sample
import sklearn
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import gensim

In [2]:
import pandas as pd
import numpy as np
from collections import defaultdict, Counter
import glob
import json
from random import sample
import sklearn

from pandarallel import pandarallel
pandarallel.initialize()

datafiles = glob.glob("/home/phadke/ONR/ONR/big_data/Twitter/*.csv")

INFO: Pandarallel will run on 128 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [3]:
plist = ['_BJP.csv', '_INC.csv', '_AAP.csv']
pcount = defaultdict(list)
for d in datafiles:
    for p in plist:
        if p in d:
            pcount[p].append(d)

In [4]:
for p in pcount.keys():
    print(p, len(pcount[p]))

_INC.csv 3006
_AAP.csv 430
_BJP.csv 3255


In [5]:
final_files = defaultdict(list)
for p in pcount:
    try:
        final_files[p] = sample(pcount[p], 1000)
    except:
        final_files[p] = sample(pcount[p], 430)

In [6]:
for p in final_files.keys():
    print(p, len(final_files[p]))

_INC.csv 1000
_AAP.csv 430
_BJP.csv 1000


## read randomly sampled data

In [7]:
rowlist = []
for p in final_files.keys():
    party = p.replace("_", "").replace(".csv", "")
    for d in final_files[p]:
        with open(d, "r") as jsonfile:
            for line in jsonfile:
                job= json.loads(line)
                if "text" in job:
                    text = job['text']
                else:
                    text = None
                if "screen_name" in job:
                    sn = job['screen_name']
                else:
                    sn = None
                    
                if "lang" in job:
                    language = job['lang']
                else:
                    language = None
                    
                row = [sn, text, party, language]
                rowlist.append(row)


In [8]:
frame = pd.DataFrame(rowlist, columns=['screen_name','text','party', 'language'])

In [9]:
frame = frame.dropna(subset=['text'])

In [10]:
frame.head()

,screen_name,text,party,language
0,None,RT @INCIndia: The North East region has been f...,INC,en
1,None,RT @INCIndia: किसी देश की राजधानी कानून की दृष...,INC,hi
2,None,अमित शाह जी आपकी सरकार ने देश से गरीबी हटाई नह...,INC,hi
3,None,कांग्रेसजनों ने मेरे नेतृत्व मे जनता को @INCIn...,INC,hi
4,None,जमालपुर चौराहा बड़ी मस्जिद से शुरू हुई कांग्रे...,INC,hi


In [11]:
def startwithRT(text):
    if text.startswith("RT "):
        return 1
    
frame['is_RT'] = frame['text'].parallel_apply(lambda x: startwithRT(x))

In [12]:
print(len(frame))
print(len(frame.loc[frame['is_RT']!=1]))

3118590
1240508


In [13]:
subframe = frame.loc[frame['is_RT']!=1]
subframe.head()

,screen_name,text,party,language,is_RT
2,None,अमित शाह जी आपकी सरकार ने देश से गरीबी हटाई नह...,INC,hi,NaN
3,None,कांग्रेसजनों ने मेरे नेतृत्व मे जनता को @INCIn...,INC,hi,NaN
4,None,जमालपुर चौराहा बड़ी मस्जिद से शुरू हुई कांग्रे...,INC,hi,NaN
5,None,ग्राम जवां सिकंदरपुर की पूर्व ब्लॉक प्रमुख रेन...,INC,hi,NaN
6,None,कांग्रेस की राष्ट्रीय महासचिव @priyankagandhi ...,INC,hi,NaN


In [14]:
from analysis_func.text_preproc import preproc_text


In [15]:
print("cleaning data")
#text preproessing - filter engligh, hindi, marathi stop words, remove puncts, hash, mentions, urls, weird spaces etc.
subframe['clean_text'] = subframe['text'].parallel_apply(lambda x: preproc_text(x))

cleaning data


<ipython-input-15-23f846663776>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subframe['clean_text'] = subframe['text'].parallel_apply(lambda x: preproc_text(x))


In [16]:
subframe.sample(5)

,screen_name,text,party,language,is_RT,clean_text
706889,6868tarkeshwar,https://t.co/MUYr9t8Y6v,INC,und,NaN,
2740216,vdsharmabjp,सभी देशवासियों को रूप चतुर्दशी 'छोटी दीपावली' ...,BJP,hi,NaN,देशवासियों रूप चतुर्दशी छोटी दीपावली
2203508,sumeetbhasin,@narendramodi Thanks to your visionary leaders...,BJP,en,NaN,visionary leadership encouragement sahib god b...
1882447,OPSharmaBJP,भारत माँ की रक्षा करते हुए बलिदान हुए सैनिकों ...,BJP,hi,NaN,माँ रक्षा बलिदान सैनिकों अवसर श्रद्धांजलि अर्प...
1836465,None,"पूर्वोत्तर का विकास, वहां की शान्ति और राजनीति...",BJP,hi,NaN,पूर्वोत्तर विकास शान्ति राजनीतिक स्थिरता हमारी...


In [17]:
# keep text only in hi, mr and en for now
keep_lang = ['en','hi','mr']
lang_frame = subframe.loc[subframe['language'].isin(keep_lang)]


In [18]:
# check_tokens = ['and', 'की', 'में']
# def checkforwords(text):
#     for c in check_tokens:
#         if c in text:
#             return 1
    

In [19]:
# lang_frame['is_trashy'] = lang_frame.parallel_apply(lambda x: checkforwords(x))
# lang_frame.loc[lang_frame['is_trashy']==1].sample(5)

In [20]:
import gensim
from gensim.models.phrases import Phrases
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel


In [21]:
print("prepping corpus for topicmodel")
# prepare data for topic modeling
corp = lang_frame['clean_text'].tolist()
parties = lang_frame['party'].tolist()
corp_tokens = [c.split() for c in corp]
phrases = Phrases(corp_tokens, min_count=5, threshold=0.1)
tokes = [phrases[t] for t in corp_tokens]


prepping corpus for topicmodel


In [22]:
BJP_data = []
INC_data = []
AAP_data = []

for i in range(len(tokes)):
    if parties[i]=='BJP':
        BJP_data.append(tokes[i])
    if parties[i]=='INC':
        INC_data.append(tokes[i])
    if parties[i]=='AAP':
        AAP_data.append(tokes[i])


In [23]:
BJP_data[89]

['honor',
 'youve',
 'invited',
 'deliver_keynote',
 'inaugural',
 'sydney_dialogue']

## phrasing based on languages

## BJP topic modeling

In [24]:
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from analysis_func.text_preproc import preproc_text
from analysis_func.topic_preproc import prepare_corp
from analysis_func.topic_model import topicmodel
from analysis_func.saveload_topicmodels import save_modelanddata, load_modelanddata


In [25]:
%%time
BJP_id2word, BJP_corpus = prepare_corp(BJP_data)
BJP_model = topicmodel(5, BJP_corpus, BJP_id2word)

CPU times: user 5min 26s, sys: 6min 52s, total: 12min 19s
Wall time: 11min 22s


In [26]:
from pprint import pprint
pprint(BJP_model.print_topics())
doc_lda = BJP_model[BJP_corpus]

[(0,
  '0.006*"राम" + 0.003*"श्रद्धांजलि" + 0.002*"झारखण्ड" + 0.002*"namo_app" + '
  '0.002*"दिवस" + 0.002*"रहें" + 0.001*"उपस्थित" + 0.001*"युवा_मोर्चा" + '
  '0.001*"महादेव" + 0.001*"पीएम"'),
 (1,
  '0.004*"आभार" + 0.002*"शुभकामनाएँ" + 0.002*"app" + 0.002*"ॐ_शांति" + '
  '0.002*"शत_शत" + 0.002*"namo_app" + 0.001*"checked" + 0.001*"अभिनंदन" + '
  '0.001*"स्वयंसेवक_संघ" + 0.001*"भोलेनाथ"'),
 (2,
  '0.011*"धन्यवाद" + 0.002*"आपकी_शुभकामनाओं" + 0.001*"साहब" + 0.001*"राजस्थान" '
  '+ 0.001*"शुभकामनाओं_धन्यवाद" + 0.001*"चाहिए" + 0.001*"होगा" + 0.001*"कभी" + '
  '0.001*"सब" + 0.001*"लोग"'),
 (3,
  '0.002*"sir" + 0.001*"namo_app" + 0.001*"wishes" + 0.001*"om_shanti" + '
  '0.001*"life" + 0.001*"()" + 0.001*"bengal" + 0.001*"दिवस" + 0.001*"team" + '
  '0.001*"सादर_प्रणाम"'),
 (4,
  '0.002*"कामना" + 0.002*"उत्तम_स्वास्थ्य" + 0.002*"namo_app" + 0.002*"पीएम" + '
  '0.002*"ईश्वर_आपके" + 0.002*"ईश्वर_प्रार्थना" + 0.002*"प्रदान" + '
  '0.002*"स्वास्थ्य_दीर्घायु" + 0.002*"आपके_उत्तम" + 0.002*"हूं"')]